In [ ]:
# Load pacakges ----
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from IPython.display import HTML
from IPython.display import Markdown as md
from IPython.core.display import HTML as Center
from config import marker, qof_measure_marker, start_date, end_date, demographics, codelist_path
from IPython.display import Image, display

from utilities import *
%matplotlib inline

In [ ]:
display(
    md(f"## Changes in {marker} between {start_date} and {end_date}"),
    md(f"### Indicator: {qof_measure_marker}")
)

In [ ]:
md(f"Below are various time-series graphs showing changes in {marker} (Indicator: {qof_measure_marker}) code use.")

In [ ]:
display(
md("### Methods"),
md(f"Using OpenSAFELY-TPP, covering 40% of England's population, we have assessed coding activity related to {marker} between {start_date} and {end_date}. The codelist used can be found here at [OpenSAFELY Codelists](https://codelists.opensafely.org/).  For each month within the study period, we have calculated the rate at which the code was recorded per 1000 registered patients."),
md(f"All analytical code and output is available for inspection at the [OpenSAFELY GitHub repository](https://github.com/opensafely)")
)

In [ ]:
codelist = pd.read_csv(f'../{codelist_path}')

image_paths = {d: f'../output/plot_{d}.png' for d in demographics}
image_paths['total'] = '../output/plot_total.png'

In [ ]:
md(f"## Total {marker} ")

In [ ]:
Image(filename=image_paths['total'])

In [ ]:
display(
    md("### Sub totals by sub codes"),
    md("Events for the top 5 subcodes across the study period")
)

In [ ]:
child_table = pd.read_csv('../output/child_code_table.csv')
child_table

In [ ]:
md("## Total Number by GP Practice")

In [ ]:
practice_table = pd.read_csv('../output/rate_table_practice.csv', parse_dates=['date']).sort_values(by='date')
percentage_practices = get_percentage_practices(practice_table)
md(f"Percentage of practices with a recording of a code within the codelist during the study period: {percentage_practices}%")

In [ ]:
Image(filename='../output/decile_chart.png')

In [ ]:
i = 0
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'") 

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")  

In [ ]:
Image(filename=image_paths[demographics[i]])

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
display(Image(filename=image_paths[demographics[i]]))

In [ ]:
i += 1
md(f"## Breakdown of {marker} by '{demographics[i]}'")

In [ ]:
Image(filename=image_paths[demographics[i]]) 